In [7]:
import pandas as pd
import sys
import os
import re
sys.path.append(os.path.abspath('../scripts'))  #esto lo colocamos para que el python nos lea la ruta bien
from utils import process_null_values # Importa la funcinn desde utils
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
dataset = pd.read_csv('../data_process/cellphones.csv')


In [8]:
dataset.head()
dataset.dtypes

overall            int64
processed_text    object
dtype: object

In [11]:
#antes de pasar a hacer la lematizacion hare una eliminacion de muchos datos positivos que hay
#eliminar muchos registros para balancear la data
condition = dataset['overall'] == 5
num_to_remove = 1000
rows_to_remove = dataset[condition].head(num_to_remove)

# Eliminar esas filas del DataFrame original
dataset = dataset.drop(rows_to_remove.index)

dataset.describe()


,overall
count,130003.000000
mean,3.701376
std,1.293913
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [12]:
analyzer = SentimentIntensityAnalyzer()

def analizar_vader(texto):
    # Obtener el puntaje de polaridad
    sentiment_score = analyzer.polarity_scores(texto)
    compound_score = sentiment_score['compound']

    # Evaluar el sentimiento en base al puntaje "compound"
    if compound_score >= 0.5:
        return 'contento'  # Muy positivo
    elif compound_score >= 0.05:
        return 'positivo'  # Positivo
    elif compound_score <= -0.8:
        return 'enojado'  # Muy negativo
    elif compound_score <= -0.5:
        return 'insatisfecho'  # Negativo pero no tan extremo como "enojado"
    elif compound_score <= -0.05:
        return 'negativo'  # Negativo
    else:
        return 'neutral'  # Neutral

In [13]:
# Verificar si hay valores nulos (NaN) en la columna
process_null_values(dataset)


Total de registros:
la columna processed_text tiene un Total de: 1 valores vacios
--------------------------------------------------------------------------------
Porcentaje de registros:
la columna processed_text tiene un Porcentaje de: 0.0007692130181611193 valores vacios
--------------------------------------------------------------------------------
Si las columnas son irrelevantes, podemos hacer algo con sus datos, que te gustaria eliminarlos o los conservamos?
todas las filas con la columna processed_text han sido eliminadas
data limpiada maestro, que tengas un buen dia :)


In [14]:
dataset['sentiment'] = dataset['processed_text'].apply(analizar_vader)

In [16]:
dataset.head(30)

,overall,processed_text,sentiment
0,4,look good stick good dont like round shape alw...,negativo
1,4,item arrive great time perfect condition howev...,contento
2,3,make use home button easy daughter like would ...,contento
3,1,work first week charge phone 20 waste money,negativo
4,1,work great first couple weeks stop completely ...,neutral
5,3,not need amps charge things like ipads hp touc...,contento
6,4,nothing special sure nice tell power lead glow...,negativo
7,4,several chargers one vehicle keep one nothing ...,positivo
8,2,disappoint 1a didnt work ipad thats get buy ch...,negativo
9,4,nice charger tell make cheaply china charge ph...,contento


In [17]:
dataset.to_csv('../data_process/cellphones.csv', index=False)
